# Procedimiento para comparar la generación de puntos cada 500 metros

In [24]:
import csv
import os
from collections import defaultdict

import decouple
import pandas as pd
import plotly.express as px

from op_processing_for_vis.config import OUTPUT_PATH, TMP_PATH
from op_processing_for_vis.utils import get_route_id_info

config = decouple.AutoConfig(' ')

In [41]:
# load processed shape
op_date = '2022-07-02'
filename = '{}.shape'.format(op_date)
shape_path = os.path.join(OUTPUT_PATH, op_date, filename)

shape_dict = defaultdict(list)
with open(shape_path, newline='', encoding='utf-8') as csvfile:
    reader = csv.DictReader(csvfile, delimiter='|')
    for row in reader:
        latitude = float(row['Latitude'])
        longitude = float(row['Longitude'])
        is_starting_segment = int(row['IsSectionInit'])
        shape_dict[row['Route']].append((longitude, latitude, is_starting_segment))

In [42]:
# load shape file processed by mauricio
route_id_info = get_route_id_info(op_date)
mauricio_shape_dict = defaultdict(list)
mauricio_shape_path = os.path.join(TMP_PATH, 'mauricio_files', '{0}.shape'.format(op_date))
with open(mauricio_shape_path, newline='', encoding='utf-8-sig') as csvfile:
    reader = csv.DictReader(csvfile, delimiter='|')
    for row in reader:
        auth_route_code = row['Route']
        longitude = float(row['Longitude'])
        latitude = float(row['Latitude'])
        is_starting_segment = int(row['IsSectionInit'])
        if is_starting_segment == 1:
            mauricio_shape_dict[auth_route_code].append((longitude, latitude))

In [51]:
def generate_shape_evaluation_map(my_points, mauricio_points, filename, auto_open=False):
    mapbox_access_token = config('MAPBOX_ACCESS_TOKEN')

    # my points
    df = pd.DataFrame(my_points, columns=['lon', 'lat', 'is_starting_segment'])
    df = df[df['is_starting_segment'] == 0]
    fig = px.line_mapbox(df, lat='lat', lon='lon', color_discrete_sequence=['green'],
                         zoom=11, height=700, title="")

    # mauricio points
    df2 = pd.DataFrame(mauricio_points, columns=['lon', 'lat'])
    fig2 = px.scatter_mapbox(df2, lat='lat', lon='lon',
                             color_discrete_sequence=['yellow'], hover_data=['lon', 'lat'])

    # my points
    df3 = pd.DataFrame(my_points, columns=['lon', 'lat', 'is_starting_segment'])
    df3 = df3[df3['is_starting_segment'] == 1]
    fig3 = px.scatter_mapbox(df3, lat='lat', lon='lon',
                             color_discrete_sequence=['blue'], hover_data=['lon', 'lat'])

    fig.add_trace(fig2.data[0])
    #fig.add_trace(fig3.data[0])

    fig.update_layout(mapbox_style="dark", mapbox_accesstoken=mapbox_access_token)
    fig.update_layout(margin={"r": 0, "t": 0, "l": 0, "b": 0})
    #fig.show()
    fig.write_html(os.path.join(TMP_PATH, 'comparacion', '{0}.html'.format(filename)), auto_open=auto_open)

In [52]:
# example
auth_route_code = 'B01 00I'
my_points = shape_dict[auth_route_code]
mauricio_points = mauricio_shape_dict[auth_route_code]

generate_shape_evaluation_map(my_points, mauricio_points, auth_route_code)

In [53]:
print('Felipe -> N° puntos cada 500 mts:', len([point for point in my_points if point[2] == 1]))
print('Mauricio -> N° puntos cada 500 mts:', len(mauricio_points))

Felipe -> N° puntos cada 500 mts: 21
Mauricio -> N° puntos cada 500 mts: 21


In [46]:
my_points

[(-70.620153, -33.400169, 1),
 (-70.620196, -33.400169, 0),
 (-70.620281, -33.40014, 0),
 (-70.620345, -33.400067, 0),
 (-70.620549, -33.400083, 0),
 (-70.621943, -33.400443, 0),
 (-70.623564, -33.400909, 0),
 (-70.624895, -33.400215, 0),
 (-70.625057, -33.400134, 1),
 (-70.626951, -33.399178, 0),
 (-70.627026, -33.399132, 0),
 (-70.627995, -33.398642, 0),
 (-70.62808, -33.398604, 0),
 (-70.627121, -33.397284, 1),
 (-70.626866, -33.396934, 0),
 (-70.626734, -33.396737, 0),
 (-70.62669, -33.396693, 0),
 (-70.626839, -33.396628, 0),
 (-70.627191, -33.396452, 0),
 (-70.629736, -33.395147, 0),
 (-70.630151, -33.394943, 0),
 (-70.630567, -33.394721, 0),
 (-70.630578, -33.394715, 1),
 (-70.631387, -33.394313, 0),
 (-70.632952, -33.39349, 0),
 (-70.63358, -33.393175, 0),
 (-70.634145, -33.392915, 0),
 (-70.635278, -33.392555, 1),
 (-70.635449, -33.392501, 0),
 (-70.637725, -33.39175, 0),
 (-70.638579, -33.391432, 0),
 (-70.6386, -33.391404, 0),
 (-70.638824, -33.391338, 0),
 (-70.638932, -33.

In [33]:
mauricio_points

[(-70.620153, -33.400169),
 (-70.62808, -33.398604),
 (-70.62669, -33.396693),
 (-70.632952, -33.39349),
 (-70.638824, -33.391338),
 (-70.64244, -33.39093),
 (-70.649889, -33.387485),
 (-70.655425, -33.38701),
 (-70.660772, -33.386552),
 (-70.664078, -33.386268),
 (-70.669701, -33.386282),
 (-70.6703, -33.383902),
 (-70.677766, -33.384199),
 (-70.680176, -33.384842),
 (-70.684968, -33.388051),
 (-70.689852, -33.389923),
 (-70.689849, -33.392548),
 (-70.699277, -33.393539),
 (-70.700886, -33.393407),
 (-70.697433, -33.386314),
 (-70.697101, -33.385631)]

In [ ]:
preguntar
a
mauricio
pq
no
calzan
calza
solo
el
punto
inicial
están
raras
sus
asignaciones
pq
no
pareciera
que
todos
tuvieran
500
metros
el también marca el punto final ¿por qué?